import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter, OrderedDict

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
cols = ['status', 'duration', 'credit_hist', 'purpose', 'credit_amt', 'savings', 'employment', 'installment_rate', 'personal_status', 'debtors', 'residencesince', 'property', 'age', 'install_plans', 'housing', 'existing_credits', 'job', 'maintenance_paying_people', 'telephone', 'foreign_worker', 'result']

In [3]:
df = pd.read_table('german.data', names=cols, sep=" ", index_col=False)

In [4]:
def preprocess_german(df): 
    df['status'] = df['status'].map({'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}).astype(int)
    df['credit_hist'] = df['credit_hist'].map({'A34': 0, 'A33': 1, 'A32': 2, 'A31': 3, 'A30': 4}).astype(int)
    df.loc[(df['credit_amt'] <= 2000), 'credit_amt'] = 0
    df.loc[(df['credit_amt'] > 2000) & (df['credit_amt'] <= 5000), 'credit_amt'] = 1
    df.loc[(df['credit_amt'] > 5000), 'credit_amt'] = 2   
    df.loc[(df['duration'] <= 12), 'duration'] = 0
    df.loc[(df['duration'] > 12) & (df['duration'] <= 24), 'duration'] = 1
    df.loc[(df['duration'] > 24) & (df['duration'] <= 36), 'duration'] = 2
    df.loc[(df['duration'] > 36), 'duration'] = 3
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['savings'] = df['savings'].map({'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}).astype(int)
    df['employment'] = df['employment'].map({'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}).astype(int)    
    df['gender'] = df['personal_status'].map({'A91': 1, 'A92': 0, 'A93': 1, 'A94': 1, 'A95': 0}).astype(int)
    df['debtors'] = df['debtors'].map({'A101': 0, 'A102': 1, 'A103': 2}).astype(int)
    df['property'] = df['property'].map({'A121': 3, 'A122': 2, 'A123': 1, 'A124': 0}).astype(int)        
    df['install_plans'] = df['install_plans'].map({'A141': 1, 'A142': 1, 'A143': 0}).astype(int)
    df['job'] = df['job'].map({'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}).astype(int)    
    df['telephone'] = df['telephone'].map({'A191': 0, 'A192': 1}).astype(int)
    df['foreign_worker'] = df['foreign_worker'].map({'A201': 1, 'A202': 0}).astype(int)
    pd.get_dummies(df, columns=['purpose', 'housing'], drop_first=True)
    
    return df


In [5]:
X = preprocess_german(df);

In [6]:
df = df.drop(["purpose", "housing", "personal_status", "result"], axis=1)

In [7]:
df.head()

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,gender
0,0,0,0,0,4,4,4,0,4,3,1,0,2,2,1,1,1,1
1,1,3,2,2,0,2,2,0,2,3,0,0,1,2,1,0,1,0
2,3,0,0,1,0,3,2,0,3,3,1,0,1,1,2,0,1,1
3,0,3,2,2,0,3,2,2,4,2,1,0,1,2,2,0,1,1
4,0,1,1,1,0,2,3,0,4,0,1,0,2,2,2,0,1,1


In [8]:
def match_pattern(string1, string2):
    if len(string1) == len(string2):
        for i in range(len(string1)):
            if string1[i].isdigit() and string2[i].isdigit():
                if string1[i] != string2[i]:
                    return 0
        else:
            return 1
        return 0

In [9]:
def create_patterns(df, threshold):
    patterns = ['XXXXXXXXXXXXXXXXXX', 
                'XXXXXXXXXXXXXXXXX0',
                'XXXXXXXXXXXXXXXX00',
                'XXXXXXXXXXXXXXXX10', 
                'XXXXXXXXXXXXXXXXX1', 
                'XXXXXXXXXXXXXXXX01', 
                'XXXXXXXXXXXXXXXX11' ]
    # Initialize a dictionary to store the pattern counts
    pattern_counts = {pattern: 0 for pattern in patterns}

    # Iterate through each row of the dataset
    for i in range(len(df)):
        # Iterate through each pattern
        for pattern in patterns:
            res = match_pattern(pattern, "".join(str(x) for x in df.iloc[i].values.tolist()))
            # Increment the count for the pattern if it is found
            if res == 1:
                pattern_counts[pattern] += 1

    # Output the pattern counts
    print('Pattern counts:')
    for pattern, count in pattern_counts.items():
        print(pattern + ": " + str(count))
        
    print('\nUncovered patterns:')
    for pattern, count in pattern_counts.items():
        if count < threshold:
            print(pattern)

        

In [10]:
create_patterns(df, 35);

Pattern counts:
XXXXXXXXXXXXXXXXXX: 1000
XXXXXXXXXXXXXXXXX0: 310
XXXXXXXXXXXXXXXX00: 7
XXXXXXXXXXXXXXXX10: 303
XXXXXXXXXXXXXXXXX1: 690
XXXXXXXXXXXXXXXX01: 30
XXXXXXXXXXXXXXXX11: 660

Uncovered patterns:
XXXXXXXXXXXXXXXX00
XXXXXXXXXXXXXXXX01


In [11]:
"".join(str(x) for x in df.iloc[0].values.tolist())

'000044404310221111'

In [12]:
res = match_pattern('xxxx0', '11110')
print(res)

1


In [13]:
for item in df: 
    print(item)

status
duration
credit_hist
credit_amt
savings
employment
installment_rate
debtors
residencesince
property
age
install_plans
existing_credits
job
maintenance_paying_people
telephone
foreign_worker
gender


In [14]:

#Return the inverted index matrix for the dataset
def preprocessing(dataset):
    
    #Get cardinalities out of the dataset
    cardinalities = []
#     cardinalities = [[0,1],[0,1],[0,1]]
    for col in dataset.columns:
        cardinalities.append(dataset[col].unique().tolist())

    for cardinality in cardinalities:
        cardinality.sort()
        
    num = 1
    for cardinality in cardinalities:
        num *= len(cardinality)+1
        
    print('Total number of patterns in the dataset: ',num)


    #Get unique value combinations count
    dataset_string = []
#     dataset_string = ['010','001','000','011','001']
    for i in range(len(dataset)):
        dataset_string.append("".join(str(x) for x in dataset.iloc[i].values.tolist()))
        
    counts = OrderedDict()

    for item in dataset_string:
        counts[item] = counts.get(item, 0) + 1

    data_unique_values = list(counts.keys())
    data_value_counts = list(counts.values())
    
    inverted_ind = []
    
    #create inverted index matrix
    for i,cardinality in enumerate(cardinalities):
        for cardinality_val in cardinality:
            new_row = []
            for val in data_unique_values:
                if cardinality_val == int(val[i]):
                    new_row.append(1)
                else:
                    new_row.append(0) 
            inverted_ind.append(new_row)
            
    return inverted_ind, data_value_counts, cardinalities
        

def cov(pattern, dataset):  #tested_OK
    """
    Returns the number of instances in the dataset covered by the given pattern.
    """
    count = 0

    for i in range(len(dataset)):
        res = match_pattern(pattern, "".join(str(x) for x in df.iloc[i].values.tolist()))
        if res == 1:
            count += 1
#     print(count)
    return count


def coverage_optimized(pattern, inverted_ind, data_value_counts, cardinalities):  #invertedindices  #tested_OK for the example given in the research paper

    result_and = [1] * len(inverted_ind[0])
    row_index = 0
    for i,x in enumerate(pattern):
        if x!= 'X':
            
            #find the value x in cardinalities[i]
            index = cardinalities[i].index(int(x))
            row_index += index
            inverted_ind_row = inverted_ind[row_index]
            for j in range(len(inverted_ind_row)):
                result_and[j] = int(inverted_ind_row[j] and result_and[j])
            row_index -= index
            row_index += len(cardinalities[i])
        else:
            row_index += len(cardinalities[i])      

    # DOT Product between the above result and the count array for the datapoints
    coverage = sum([x*y for x, y in zip(data_value_counts, result_and)])
    return coverage
    

def generate_parent_nodes(pattern):  #tested_OK
    """
    Generates all parent nodes of the given pattern by replacing one deterministic
    cell with a wildcard character.
    """
    parents = []
    for i in range(len(pattern)):
        new_string = pattern[:i] + "X" + pattern[i+1:]
        if new_string != pattern:
            parents.append(new_string)
    return parents


def generate_nodes(pattern, cardinalities): #tested_OK
    """
    Generates all nodes on the given pattern and cardinalities based on Rule 1.
    """
    
    """
    TODO: Make cardinalities 2D vector, so that it can be traversed to find the children of a pattern
    """

    # Find the index of the right-most deterministic element in the pattern
    index = len(pattern) - 1
    rm_deter = -1
    while index >= 0:
        if pattern[index] != 'X':
            rm_deter = index
            break
        index -= 1

    candidate_nodes = []
    rm_deter += 1
    if rm_deter >= 0:
        while rm_deter < len(pattern):
            index = rm_deter
            for value in cardinalities[index]:
                candidate_node = pattern
                candidate_node = pattern[:index] + str(value) + pattern[index+1:] 
                candidate_nodes.append(candidate_node)
            rm_deter += 1

    return candidate_nodes


def dominance(pattern, mups):
    #iterate through the mups, find if any of the mups is an ancestor for the pattern p return 0
    # if pattern p is an ancestor for any of the mups return 1
    # else return -1
    
    for m in mups:
        for i,x in enumerate(m):
            if m[i] != pattern[i]:
                if m[i] == 'X' and pattern[i] != 'X':
                    return 0
                elif m[i] != 'X' and pattern[i] == 'X':
                    return 1
    
    return -1

    
def deepdiver(dataset, threshold):
    """
    Finds the maximal uncovered patterns in the dataset.
    """ 
    inverted_index, unique_value_counts, cardinalities = preprocessing(dataset);

    stack = ['XXXXXXXXXXXXXXXXXX']
    maximal_uncovered = []
    while stack:
        pattern = stack.pop()
        uncovered_flag = False
        if dominance(pattern, maximal_uncovered) == 0:
            continue
        elif dominance(pattern, maximal_uncovered) == 1:
            uncovered_flag = False
        else: 
            count = coverage_optimized(pattern, inverted_index, unique_value_counts, cardinalities)
            if count < threshold:
                uncovered_flag = True
        if uncovered_flag:
            stack0 = []
            stack0.append(pattern)
            while stack0:
                pattern0 = stack0.pop()
                parent_nodes = generate_parent_nodes(pattern0)
                for p in parent_nodes:
                    count0 = coverage_optimized(p, inverted_index, unique_value_counts, cardinalities )
                    if count0 < threshold:
                        stack0.append(p)
                        break                   
                maximal_uncovered.append(pattern)
        else:
            stack.extend(generate_nodes(pattern, cardinalities))
            
    print('MUPs are: ', maximal_uncovered)
    return maximal_uncovered


In [15]:
#     #MUPs
#     print('MUP coverage')
#     print(coverage_optimized('XXXXXXXXXXXXXXX101', inverted_index, unique_value_counts, cardinalities))
#     print(coverage_optimized('XXXXXXXXXXXXXXX100', inverted_index, unique_value_counts, cardinalities))
    
#     #MUP Parents
#     print('MUP Parents coverage')
#     print(coverage_optimized('XXXXXXXXXXXXXXXX01', inverted_index, unique_value_counts, cardinalities))
#     print(coverage_optimized('XXXXXXXXXXXXXXXX00', inverted_index, unique_value_counts, cardinalities))
#     print(coverage_optimized('XXXXXXXXXXXXXXX1X1', inverted_index, unique_value_counts, cardinalities))
#     print(coverage_optimized('XXXXXXXXXXXXXXX1X0', inverted_index, unique_value_counts, cardinalities))
    
#     #MUP Children
#     print('MUP children coverage')
#     print(coverage_optimized('X0XXXXXXXXXXXXX101', inverted_index, unique_value_counts, cardinalities))
#     print(coverage_optimized('X01XXXXXXXXXXXX100', inverted_index, unique_value_counts, cardinalities))
#     print(coverage_optimized('3XXXXXXXXXXXXXX101', inverted_index, unique_value_counts, cardinalities))
#     print(coverage_optimized('2XXXXXXXXXXXXXX100', inverted_index, unique_value_counts, cardinalities))

In [16]:
inv, d, cardinalities = preprocessing(df)
print(coverage_optimized('3XXXXXXXXXXX1XXX11',inv,d,cardinalities))


Total number of patterns in the dataset:  196830000000
140


In [17]:
# df1 = [[0,1,0],[0,0,1],[0,0,0],[0,1,1],[0,0,1]]
# deepdiver(df1, 1)

In [30]:
deepdiver(df, 7)

Total number of patterns in the dataset:  196830000000
MUPs are:  ['XXXXXXXXXXXXXXX10X']


['XXXXXXXXXXXXXXX10X']

In [19]:
def getStringsMatchingWithPattern(pattern, df):
    
    
    #Get unique value combinations count
    dataset_string = []
    for i in range(len(df)):
        dataset_string.append("".join(str(x) for x in df.iloc[i].values.tolist()))
        
    
    strings = []
    flag = True
    for s in dataset_string:
        flag = True
        for i,x in enumerate(s):
            if x != pattern[i]:
                if pattern[i] != 'X':
#                     print('string:',s[i])
#                     print('string:',pattern[i])
                    flag = False
                    
        if flag:
            strings.append(s)
            
    print(len(strings))       
    return strings
        
            

In [20]:
print(getStringsMatchingWithPattern('XXXXXXXXXXXXXXX101', df))

3
['302002104200121101', '012204104210131101', '302002202300121101']


In [21]:
print(getStringsMatchingWithPattern( 'XXXXXXXXXXXXXXX100', df))

2
['012241102200121100', '134202102201131100']


In [22]:
print(getStringsMatchingWithPattern( 'XXXXXXXXXXXXXXXX0X', df))

37
['000101103310212001', '300142201100221001', '012241102200121100', '000141213300221000', '202031202300101001', '302002104200121101', '000014324310222001', '102002422300121001', '302033302210121001', '304012401001121001', '300004304300212001', '012204104210131101', '312143403200121001', '302002202300121101', '302044412210111001', '202041102300112001', '000103104300211001', '102002102100121000', '012001402300121001', '302022302000112001', '302012302300111001', '000102302300212001', '000003403200221000', '000001204300112001', '013102124101121000', '102003423300221001', '200002101310302001', '102001303300111001', '010102422201221001', '302112122300122001', '200004104310222001', '000102404201211001', '300033302200121000', '012143212300111001', '134202102201131100', '012001411200111001', '302002302300112001']


In [23]:
print(getStringsMatchingWithPattern( 'XXXXXXXXXXXXXXXX1X', df))

963
['000044404310221111', '132202202300121010', '300103203310112011', '032203224210122011', '011102304010222011', '322242204000112111', '312124304210121011', '122202202100131111', '302133204310111011', '120200402100231011', '102001301100121010', '032101304200121010', '102002101100121110', '010004404110211011', '012002204100121010', '012012402100111010', '310144404210221011', '024241203101321011', '112204402000131110', '312124302100122111', '300102404110321111', '002122203300122011', '100011304200121011', '002002102301111111', '304004404100111011', '203032303300221010', '102102322300121011', '031204304010221111', '112031303301121111', '012102202101121011', '112212202100221111', '300044404010111011', '202001401201131110', '130101402200211011', '330202103001121111', '212102412301121011', '202002202100121111', '102002403300121011', '322124402101131011', '102022301100121111', '111202204300112111', '020213104100221011', '030204404011211010', '300001404100231010', '322124404100121011', '0020